# RAG using watsonx

## 1. Importing Libraries

In [1]:
# !pip install ibm-generative-ai

In [2]:
import os
from dotenv import load_dotenv
import os

from genai.extensions.langchain import LangChainInterface
from genai.schemas import GenerateParams
from genai.credentials import Credentials

from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

## 2. Setting up the API key and URL

In [3]:
load_dotenv()
api_key = os.getenv("GENAI_KEY", None)
api_url = os.getenv("GENAI_API", None)
creds = Credentials(api_key, api_endpoint=api_url)

## 3. Defining the model and parameters

In [4]:
model_id = "meta-llama/llama-2-70b-chat"

In [5]:
params = GenerateParams(
            decoding_method="greedy",
            max_new_tokens=1000,
            min_new_tokens=200,
            temperature=0.7,
        )

## 4. Creating the langchain interface

In [6]:
llm = LangChainInterface(model=model_id, params=params, credentials=creds)

## 5. Loading the pdf data and storing it in vector database

In [7]:
def load_pdf(): 
    pdf_name = 'test.pdf'
    loaders = [PyPDFLoader(pdf_name)]

    index = VectorstoreIndexCreator(
        embedding = HuggingFaceEmbeddings(model_name='all-MiniLM-L12-v2'), 
        text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    ).from_loaders(loaders)

    return index

In [8]:
index = load_pdf()

/Users/charanahu/VS_Code/YouTube/watsonxRAG/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 6. Setting a chain using langchain

In [9]:
chain = RetrievalQA.from_chain_type(llm=llm, 
                                    chain_type='stuff', 
                                    retriever=index.vectorstore.as_retriever(), 
                                    input_key='question')

In [10]:
response = chain.run("PROMPT OPTIMIZATION EXPERIMENTS results?")

In [11]:
print(response)

 The results of the prompt optimization experiments are shown in Table 10, 11, 12, and 13, and visualized in Figure 23. The optimizations find instructions better than the empty starting point, and most of the found instructions are better than “Let’s think step by step”. However, the sensitivity of model performance to subtle changes in the instruction is a challenge, and the optimization stability can be improved by generating multiple instructions at each step. Different optimizer LLMs produce instructions of different styles. The results on more BBH tasks are deferred to Appendix E.

In this case, the answer is found in the last paragraph of section 5.2.3. The answer references tables and figures that contain the results of the prompt optimization experiments. The answer also mentions a challenge related to the sensitivity of model performance to subtle changes in the instruction, and how generating multiple instructions at each step can improve the optimization stability.
